In [ ]:
import tuxml
import pandas as pd 

df = tuxml.load_dataset()
df.replace(2, 0, inplace=True)

In [ ]:
df.sort_values("vmlinux")[:5]

In [ ]:
df_importance = pd.read_csv("feature_importanceRF.csv", header=None)
df_importance.rename(columns={0 : 'option', 1 : 'importance' }, inplace=True)
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
df_importance = df[df_importance["option"].values] # [:100]
df_importance.head()
# df_importance

In [ ]:
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]


In [ ]:
additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]
# very ad-hoc, not consistent with feature importance
# df.drop(columns=additional_ft, inplace=True) # should be a parameter of the experiment (whether or not we keep nbyes, etc.)

In [ ]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({"error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [ ]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [ ]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

target = "vmlinux"
paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
               'max_iter': [1, 5, 10, 100, 1000],}
paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
              'l1_ratio': [0.0, 0.5, 1.0, 0.1],
              'max_iter': [1, 5, 10, 100, 1000],}

for ft_dummyfication in [False]: # [False, True]:
    if ft_dummyfication:
        #Creating dummy variables in pandas
        df = pd.get_dummies(df, columns=list(df.drop(columns=["cid"]).drop(columns=size_methods).drop(columns=additional_ft).columns.values))
        df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=["nbyes", "nbno", "nbyesmodule"]).columns.values))
        
    ft_selection =  {'None': df.drop(columns=["cid"]).drop(columns=size_methods).columns,
                    # 'RandomForest': df_importance.columns[:100],
                     # 'RandomForest200': df_importance.columns[:200],
                     # 'RandomForest1000': df_importance.columns[:1000],
                    }
        
    for origin_ft_selection in ft_selection:
        print(origin_ft_selection)
        # for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        for size in [0.9]:
            print(size)

            x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
            # x_train, y_train = np.array(x_train.values), np.array(y_train.values)

            model = {# 'LinearRegression': LinearRegression(),
                    # 'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
#                     'LassoCV': linear_model.LassoCV(cv=2).fit(x_train, y_train),                    
                     # 'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                     # 'RidgeCV': linear_model.RidgeCV(cv=2).fit(x_train, y_train),  
                    # 'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                    'ElasticNetCV': linear_model.ElasticNetCV().fit(x_train, y_train),
                   # 'PolynomialRegression': LinearRegression()
                    }
        
            for key in model:
                if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                    print("It doesn't scale")
                else:
                    if (key == 'PolynomialRegression'):
                        #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                        pf = PolynomialFeatures(degree=2, include_bias=False)
                        x_train_pf = pf.fit_transform(x_train)
                        x_test_pf = pf.fit_transform(x_test)
                        res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train_pf, x_test=x_test_pf)
                        # model[key].coef_ = model[key].coef_[:100]
                        importanceSeries = pd.Series(model[key].coef_, index=pf.get_feature_names(x_train.columns))
                        coef_order = importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)
                    else:
                        res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)
                        coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                   
                        
                    df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, additional_ft, size, coef_order, res_model]

df_result.to_csv("results.csv", header=True)

In [ ]:
#df_result['coef_order']

In [ ]:
res = pd.read_csv('results.csv')
res

In [ ]:
# oblige to reparse 
coeffs = res.iloc[0]['coef_order']
coef = pd.DataFrame(columns=['option', 'coefficient'])
i = 0
coeffs.splitlines()
for coeff in coeffs.splitlines():
    print(coeff)
    optcoeff = [c for c in coeff.split(' ') if c is not '']
    if len(optcoeff) == 1:
        continue
    elif len(optcoeff) == 3:
        opt = optcoeff[0] + " " + optcoeff[1]
        coeff = optcoeff[2]
    else: # standard: one option, one coeff
        opt = optcoeff[0]
        coeff = optcoeff[1]
    coef.loc[i] = (opt, coeff)
    i = i + 1
coef

In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = [order_lr, order_lasso, order_ridge, order_en]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()

In [ ]:
res.iloc[1]['hyperparameters']

In [ ]:
# 'nbyes' in coef['option']